<a href="https://colab.research.google.com/github/kgovindaraju123/AIML/blob/master/NLPSimpsons.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!python -m spacy download en_core_web_lg
!python -m spacy download en

     |████████████████████████████████| 826.9MB 1.3MB/s 
  Created wheel for en-core-web-lg: filename=en_core_web_lg-2.1.0-cp36-none-any.whl size=828255078 sha256=70fc88d8b5f8146b17616a34756111dcdba1062258e54cb0c1c5a87d13f7d2d0
  Stored in directory: /tmp/pip-ephem-wheel-cache-6jiq_xwv/wheels/b4/d7/70/426d313a459f82ed5e06cc36a50e2bb2f0ec5cb31d8e0bdf09
Successfully built en-core-web-lg
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_lg')
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')
✔ Linking successful
/usr/local/lib/python3.6/dist-packages/en_core_web_sm -->
/usr/local/lib/python3.6/dist-packages/spacy/data/en
You can now load the model via spacy.load('en')


In [0]:
import re 
import pandas as pd
from time import time
from collections import defaultdict
import spacy
import logging
logging.basicConfig(format="%(levelname)s - %(asctime)s:%(message)s",datefmt='%H:%M:%S',level=logging.INFO)

In [5]:
df= pd.read_csv("simpsons_dataset.csv")
df.head(3)

,raw_character_text,spoken_words
0,Miss Hoover,"No, actually, it was a little of both. Sometim..."
1,Lisa Simpson,Where's Mr. Bergstrom?
2,Miss Hoover,I don't know. Although I'd sure like to talk t...


In [6]:
#Verify Null
df.isnull().sum()
df.shape

(158314, 2)

In [7]:
#Remove Null
df=df.dropna().reset_index(drop=True)
df.isnull().sum()

raw_character_text    0
spoken_words          0
dtype: int64

In [29]:
#Check again shape
df.shape

(131853, 2)

In [0]:
#Preprocessing part cleaning, Bigrams and most Frequent Words
nlp=spacy.load('en_core_web_sm',disable=['ner','parser'])
def cleaning(doc):
  txt=[token.lemma_ for token in doc if not token.is_stop]
  if len(txt)>2:
    return ' '.join(txt)

In [0]:
#Remove non alphabetic characters
brief_cleaning=(re.sub("[^A-Za-z']+", ' ',str(row)).lower() for row in df['spoken_words'])


In [32]:
type(brief_cleaning)

generator

In [33]:
#Define the pipeline
t=time()
txt=[cleaning(doc) for doc in nlp.pipe(brief_cleaning,batch_size=5000,n_threads=-1)]
print('Time to clean up everything:{}mins'.format(round((time()-t)/60,2)))

Time to clean up everything:1.58mins


In [34]:
type(txt)

list

In [35]:
#DF to remove missing values and duplicates
df_clean = pd.DataFrame({'Clean':txt})
df_clean = df_clean.dropna().drop_duplicates()
df_clean.shape

(85962, 1)

In [0]:
#Bigrams - Gensim phases package automatically detect common phrases (bigrams=two word sentences) from a list of Sentences
#Main reason to use bigram is to catch words like "Mr_burns" or "bart_simpson"
from gensim.models.phrases import Phrases,Phraser

In [0]:
#Phrases takes a list of words as input
sent=[row.split() for row in df_clean['Clean']]


In [0]:
#out put the sentences in DataFrame
sent

In [46]:
#creates the relvant phrases from list of sentences
#progress per - write logs for every n sentences
phrases=Phrases(sent,min_count=30,progress_per=10000)


INFO - 04:38:15:collecting all words and their counts
INFO - 04:38:15:PROGRESS: at sentence #0, processed 0 words and 0 word types
INFO - 04:38:16:PROGRESS: at sentence #10000, processed 63561 words and 52712 word types
INFO - 04:38:16:PROGRESS: at sentence #20000, processed 130943 words and 99633 word types
INFO - 04:38:16:PROGRESS: at sentence #30000, processed 192972 words and 138210 word types
INFO - 04:38:16:PROGRESS: at sentence #40000, processed 249843 words and 172231 word types
INFO - 04:38:16:PROGRESS: at sentence #50000, processed 311269 words and 208040 word types
INFO - 04:38:16:PROGRESS: at sentence #60000, processed 373589 words and 243067 word types
INFO - 04:38:16:PROGRESS: at sentence #70000, processed 436445 words and 278006 word types
INFO - 04:38:16:PROGRESS: at sentence #80000, processed 497905 words and 311099 word types
INFO - 04:38:16:collected 329881 word types from a corpus of 537154 words (unigram + bigrams) and 85962 sentences
INFO - 04:38:16:using 329881 c

gensim.models.phrases.Phrases

In [48]:
type(phrases)

gensim.models.phrases.Phrases

In [47]:
#Goal of phraser() is to cut down memeory consumption of Phrases()
bigram = Phraser(phrases) #we cant see as it is a black box process

INFO - 04:38:45:source_vocab length 329881
INFO - 04:38:48:Phraser built with 126 phrasegrams


In [61]:
#transform the corpus based on the bigrams detected
sentences=bigram[sent]


/usr/local/lib/python3.6/dist-packages/gensim/models/phrases.py:598: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


In [67]:
#Example for Gensim Model Phrases
documents = ["Mr_Raju_Govind""the mayor of new york was there" "machine learning can be useful sometimes"," new york mayor was present"]
sentence_stream=[doc.split(" ")for doc in documents]
print(sentence_stream)
bigram=Phrases(sentence_stream,min_count=1,threshold=2)

INFO - 05:03:47:collecting all words and their counts
INFO - 05:03:47:PROGRESS: at sentence #0, processed 0 words and 0 word types
INFO - 05:03:47:collected 29 word types from a corpus of 18 words (unigram + bigrams) and 2 sentences
INFO - 05:03:47:using 29 counts as vocab in Phrases<0 vocab, min_count=1, threshold=2, max_vocab_size=40000000>


[['Mr_Raju_Govindthe', 'mayor', 'of', 'new', 'york', 'was', 'theremachine', 'learning', 'can', 'be', 'useful', 'sometimes'], ['', 'new', 'york', 'mayor', 'was', 'present']]


In [68]:
#threshold - the minimum score for a bigram to be taken into account
sents=[u'Mr_Raju_Govind',u'the',u'mayor',u'of',u'the',u'new',u'york',u'was',u'there']
print(bigram[sents])

['Mr_Raju_Govind', 'the', 'mayor', 'of', 'the', 'new_york', 'was', 'there']


/usr/local/lib/python3.6/dist-packages/gensim/models/phrases.py:598: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


In [0]:
#Most Frequent words sanity check of effectiveness of Lemmatization
word_freq =defaultdict(int)
for sent in sentences:
  for i in sent:
    word_freq[i]+=1
    len(word_freq)

In [70]:
word_freq

defaultdict(int,
            {'_': 1,
             'a': 2,
             'e': 3,
             'k': 1,
             'm': 1,
             'n': 2,
             'o': 2,
             'p': 1,
             'r': 3,
             's': 2,
             't': 1,
             'w': 2,
             'y': 2})

In [65]:
#display 10 words in sort
sorted(word_freq,key=word_freq.get,reverse=True)[:11]

['e', 'r', 'n', 'w', 'y', 'o', 'a', 's', '_', 'k', 'm']

In [0]:
End of preprocessing step

In [0]:
#Training th model GENSIM word2vec implementation
import multiprocessing
from gensim.models import Word2Vec

In [95]:
#to process the data speed, let us define cores
cores=multiprocessing.cpu_count() # count the noof cores in a computer)
cores

2

In [0]:
#Define the Word2vec model
w2v_model=Word2Vec(min_count=20,window=2,size=300,sample=6e-5,alpha=0.03,min_alpha=0.0007,negative=20,workers=cores-1)
#min_count=ignoresall words with total frequency lower than value
#window = max distance between the current and predicted word within a sentence
#size =Dimensionality of feature vectors
#alpha = initial learning rate
#negative - negative sampleing will b eused how many noisy words are to be drawn
#worker - use threads to train the model

In [98]:
#Build the vocabulary table
t=time()
w2v_model.build_vocab(sentences,progress_per=10000)
print('Time to build vocabulary:{}mins'.format(round((time()-t)/60,2)))

INFO - 05:32:28:collecting all words and their counts
WARNING - 05:32:28:Each 'sentences' item should be a list of words (usually unicode strings). First item here is instead plain <class 'str'>.
INFO - 05:32:28:PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
INFO - 05:32:28:collected 13 word types from a corpus of 23 raw words and 4 sentences
INFO - 05:32:28:Loading a fresh vocabulary
INFO - 05:32:28:effective_min_count=20 retains 0 unique words (0% of original 13, drops 13)
INFO - 05:32:28:effective_min_count=20 leaves 0 word corpus (0% of original 23, drops 23)
INFO - 05:32:28:deleting the raw counts dictionary of 13 items
INFO - 05:32:28:sample=6e-05 downsamples 0 most-common words
INFO - 05:32:28:downsampling leaves estimated 0 word corpus (0.0% of prior 0)
INFO - 05:32:28:estimated required memory for 0 words and 300 dimensions: 0 bytes
INFO - 05:32:28:resetting layer weights


Time to build vocabulary:0.0mins


In [0]:
#Training model
t=time()
w2v_model.train(sentences,total_examples=w2v_model.corpus_count,epochs=30,report_delay=1)
print('Time to train the model:{}mins'.format(round((time()-t)/60,2)))
#Total examples - count the sentences

In [0]:
#Use init_sims() make the model much more effiecient 
w2v_model.init_sims(relace=True)

In [0]:
#Exploring the model
#Ask our model to find the word most similar to some of most iconoc character
w2v_model=wv.most_similar(positive=["homer"])

In [0]:
#bigram
w2v_model=wv.most_similar(positive=["homer_simpson"])